In [35]:
import chess
import numpy as np
import tensorflow as tf
import os
from tqdm import tqdm
from collections import defaultdict

import pandas as pd

In [2]:
deepchess = tf.keras.models.load_model('../models/deepchess/deepchess')

In [3]:
def fen_to_bitboard(fen):
    '''
    Takes in a FEN position and returns a bitboard notation
    '''
    
    chess_pieces = {
        'p': 0,
        'n': 1,
        'b': 2,
        'r': 3,
        'q': 4,
        'k': 5,
        'P': 6,
        'N': 7,
        'B': 8,
        'R': 9,
        'Q': 10,
        'K': 11
    }
    
    bitboard = [0]*773
    currIndex = 0
    [position, turn, castling, _, _, _] = fen.split(' ')
    for ch in position:
        if ch == '/':
            continue
        elif ch >= '1' and ch <= '8':
            currIndex += (ord(ch) - ord('0')) * 12
        else:
            bitboard[currIndex + chess_pieces[ch]] = 1
            currIndex += 12
    bitboard[768] = 1 if turn == 'w' else 0
    bitboard[769] = 1 if 'K' in castling else 0
    bitboard[770] = 1 if 'Q' in castling else 0
    bitboard[771] = 1 if 'k' in castling else 0
    bitboard[772] = 1 if 'q' in castling else 0
    return bitboard

In [4]:
STS_DIR = '../data/STS/'
count  = 0
total = 0
move_score = defaultdict(lambda: defaultdict(int))
predictors = defaultdict(list)
game_list = defaultdict(set)

for f in tqdm(os.listdir(STS_DIR)):
    file = open(STS_DIR+f)
    for game in file.readlines():
        
        fen = [g.strip() for g in game.split('-')][0]

        # find all moves
        for i in game.split('"'):
            if '=' in i:
                moves = [k.split('=') for k in [j.strip() for j in i.split(',')]]
                break
        # There are 23 games without the term c0, and all 23 only has 1 best move.
        if 'c0' not in game:
            moves = [[game.split('bm')[-1].split(';')[0].strip(), 10]]
        count+=1
        board = chess.Board(fen)
        legal_moves = list(board.legal_moves)
        total += len(legal_moves)
        for m in legal_moves:
            board = chess.Board(fen)
            board.push(m)
            predictors[count].append(fen_to_bitboard(board.fen()))
            game_list[count].add(tuple(fen_to_bitboard(board.fen())))


        for m in moves:
            board = chess.Board(fen)
            to_move, score = m[0], m[1]
            board.push_san(to_move)
            # df['id'].append(count)
            # df['bitboard'].append(fen_to_bitboard(board.fen()))
            # df['score'].append(score)
            move_score[count][tuple(fen_to_bitboard(board.fen()))] = score       
count

100%|██████████| 15/15 [00:23<00:00,  1.54s/it]


1500

In [17]:
best = {}
for k in tqdm(predictors):
    x1 = np.array([predictors[k][0]])[:, np.newaxis, :]
    for i in range(1, len(predictors[k])):
        x2 = np.array([predictors[k][i]])[:, np.newaxis, :]
        pred = deepchess.predict((x1,x2))
        # meaning x2 is preferred over x1
        if np.argmax(pred[0][0]):
            x1 = x2
    best[k] = x1

100%|██████████| 1500/1500 [49:04<00:00,  1.96s/it]


In [19]:
# 1160  out of 15,000.... tragic i know.
score = 0
for k in best:
    score += int(move_score[k][tuple(best[k][0][0])])
score

1160

In [21]:

# total number of nodes
d = 0
for k in game_list:
    d += len(game_list[k])
d

57471

In [22]:
# average nodes searched
57471/1500

38.314

In [73]:
STS_DIR = '../data/STS/'
count  = 0
total = 0
move_score = defaultdict(lambda: defaultdict(int))
predictors = defaultdict(list)
game_list = defaultdict(set)

df = defaultdict(list)

for f in tqdm(os.listdir(STS_DIR)):
    file = open(STS_DIR+f)
    for game in file.readlines():
        
        fen = [g.strip() for g in game.split('-')][0]

        # find all moves
        for i in game.split('"'):
            if '=' in i:
                moves = [k.split('=') for k in [j.strip() for j in i.split(',')]]
                break
        # There are 23 games without the term c0, and all 23 only has 1 best move.
        if 'c0' not in game or len(moves)==1:
            continue
        count+=1
        for m in moves:
            board = chess.Board(fen)
            to_move, score = m[0], m[1]
            board.push_san(to_move)
            df['id'].append(count)
            df['bitboard'].append(fen_to_bitboard(board.fen()))
            df['score'].append(score)
count

100%|██████████| 15/15 [00:01<00:00, 11.84it/s]


1381

In [74]:
pos_df = pd.DataFrame(df)
score2 = 0
for id in tqdm(pos_df['id'].unique()):
    cur_df = pos_df.loc[pos_df['id']==id].reset_index()
    x1 = np.array([list(cur_df['bitboard'])[0]])[:, np.newaxis, :]

    for i in range(1, len(cur_df)):
        x2 = np.array([list(cur_df['bitboard'])[i]])[:, np.newaxis, :]
        pred1 = deepchess.predict((x1,x2))
        
        if np.argmax(pred1[0][0]) == 1:
            x1 = x2
    
    for i in range(len(cur_df)):
        if (list(cur_df['bitboard'])[i] == x1).all():
            score2 += int(cur_df.iloc[i]['score'])
            break

100%|██████████| 1381/1381 [03:05<00:00,  7.46it/s]


In [85]:
# score, total maximum score
score2, len(pos_df['id'].unique())*10

(8546, 13810)

In [86]:
(8546/13810)*100

61.88269370021723

In [83]:
min_score = 0
for id in tqdm(pos_df['id'].unique()):
    cur_df = pos_df.loc[pos_df['id']==id].reset_index()
    min_score += min(cur_df['score'].astype('int'))
min_score

100%|██████████| 1381/1381 [00:01<00:00, 1045.73it/s]


5084